## Loading data and initializing parameters

In [1]:
from policies.network_policies.adaptive_lead_cache import AdaptiveLeadCache
from policies.network_policies.decentralized.network_lfu import NetworkLFU
from policies.network_policies.decentralized.network_lru import NetworkLRU
from policies.network_policies.lead_cache import LeadCache
from policies.network_policies.solvers.lead_cache_solver import get_opt_lead_cache, LeadCacheSolverParams
import random

import numpy as np

from utilities import display_single_level_statistics
from simulation.simulation_parameters import SimulationParameters
from simulation.simulation_statistics import SimulationStatistics
from simulation.simulation_runner import SimulationRunner
from policies.ftpl_policy import FTPLPolicy
from policies.lfu_policy import LFUPolicy
from policies.lru_policy import LRUPolicy
from policies.policy import Policy
from typing import List
from data.path import DataPath
from utilities import display_multi_level_statistics
from data.loaders import Dataset, load_bipartite_traces
from data.loaders import load_online_cache_trace
from factories.cache_factory import get_expert_ftpl_policy
from data.loaders import load_movielens
from factories.cache_factory import get_client_cache_connections



## Defining single cache simulations

In [2]:
from factories.cache_factory import get_expert_iawm_policy


def run_single_cache_simulation(ds: Dataset, cache_size: int):
    policies: List[Policy] = [
        LRUPolicy(cache_size),
        LFUPolicy(cache_size),
        FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.0008),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.0009),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.001),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.002),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.003),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.004),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.01),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.02),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.06),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.07),
        # FTPLPolicy(cache_size, ds.catalog_size, ds.trace.size, discount_rate=1.1),
        get_expert_iawm_policy(cache_size, ds.catalog_size, ds.trace.size, discount_rates=list(np.arange(-2, 1.3, 0.1))),
    ]

    runner = SimulationRunner(threads=len(policies))
    parameters = SimulationParameters(ds.trace, policies)
    statistics: List[SimulationStatistics] = runner.run_simulations(parameters)
    display_single_level_statistics(statistics)

## Define bipartite simulation

In [3]:
from policies.network_policies.network_policy import NetworkPolicy
from data.loaders import BiPartiteDataset


def run_multi_cache_simulation(
        policies: List[NetworkPolicy],
        data: BiPartiteDataset
):
    runner = SimulationRunner(threads=min([len(policies), 10]))
    statistics = runner.run_bipartite_simulations(policies, data)
    display_multi_level_statistics(statistics)

## Run single cache simulation

In [4]:
# single_cache_size = 10
# datasets: List[Dataset] = [
#     # load_movielens(DataPath.MOVIE_LENS, trace_length=100_000, cache_size=single_cache_size)
#     # load_online_cache_trace(DataPath.OSCILLATOR),
#     # load_online_cache_trace(DataPath.CHANGING_OSCILLATOR),
#     # load_online_cache_trace(DataPath.CHANGING_POPULARITY_CATALOG),
#     # load_online_cache_trace(DataPath.PSN_CATALOG),
#     # load_online_cache_trace(DataPath.FIXED_POPULARITY_CATALOG),
#     load_online_cache_trace(DataPath.SN_OSCILLATOR)
# ]
#
# for dataset in datasets:
#     print(
#         f'=========== '
#         f'{dataset.name.split("/")[-1].split(".")[0]},'
#         f' cache at {round(100 * single_cache_size / dataset.catalog_size, 1)}%'
#         f' ==========='
#     )
#     run_single_cache_simulation(dataset, single_cache_size)

## Run bipartite simulation

In [ ]:
data = load_bipartite_traces() #  T=1500, cat=50
cache_size = 5
clients = len(data.traces)
caches = 5
d_regular_degree = 2
client_cache_connections = get_client_cache_connections(clients, caches, d_regular_degree)
network_caches = [
    LeadCache(
        caches,
        client_cache_connections,
        data.catalog_size,
        d_regular_degree,
        cache_size
    ),
    AdaptiveLeadCache(
        caches,
        client_cache_connections,
        data.catalog_size,
        d_regular_degree,
        cache_size,
        list(np.arange(0.99, 1.01, 0.0001)),
        data.traces[0].size
    ),
    NetworkLFU(
        caches,
        client_cache_connections,
        data.catalog_size,
        cache_size
    ),
    NetworkLRU(
        caches,
        client_cache_connections,
        data.catalog_size,
        cache_size
    )
]
run_multi_cache_simulation(network_caches, data)